## Feature extraction postnatal development - porespy

The workflow to demonstrate the feature extraction with the function `regionprops_3D` from `porespy` was already demonstrated [here]().

This notebook aims to extract the same features from all images of the postnatal development with a for loop.

In [1]:
import apoc
import numpy as np
import os
import pandas as pd
import pyclesperanto_prototype as cle

from porespy.metrics import regionprops_3D
from skimage.measure import regionprops_table

import sys
sys.path.append("../..")
from quapos_lm import rescale_image, rescale_segmentation, predict_image

In [3]:
# Load classifier
quapos_lm = apoc.ObjectSegmenter(opencl_filename = "../../01-training-and-validation/02-quapos-lm.cl")
quapos_lm.feature_importances()

{'gaussian_blur=1': 0.32557488170342097,
 'difference_of_gaussian=1': 0.4231073391932076,
 'laplace_box_of_gaussian_blur=1': 0.25131777910337144}

In [2]:
# Define directory for images
images = "../../data/02-data-for-pixel-classifier/statistics-data/"

# Define file list
file_list = os.listdir(images)

# Show file list
print(file_list)

['C1-p08-20x-zoom-1.2-replicate-1.tif', 'C1-p08-20x-zoom-1.5-replicate-1.tif', 'C1-p08-20x-zoom-1.7-replicate-1.tif', 'C1-p08-20x-zoom-2.2-replicate-2.tif', 'C1-p08-20x-zoom-2.5-replicate-2.tif', 'C1-p08-20x-zoom-2.7-replicate-2.tif', 'C1-p08-20x-zoom-3.2-replicate-3.tif', 'C1-p08-20x-zoom-3.5-replicate-3.tif', 'C1-p08-20x-zoom-3.7-replicate-3.tif', 'C1-p08-20x-zoom-4.2-replicate-4.tif', 'C1-p08-20x-zoom-4.5-replicate-4.tif', 'C1-p08-20x-zoom-4.7-replicate-4.tif', 'C1-p10-20x-zoom-1.1-suse-replicate-5.tif', 'C1-p10-20x-zoom-1.3-suse-replicate-5.tif', 'C1-p10-20x-zoom-1.4-suse-replicate-5.tif', 'C1-p10-20x-zoom-2.2-suse-replicate-6.tif', 'C1-p10-20x-zoom-2.3-suse-replicate-6.tif', 'C1-p10-20x-zoom-2.5-suse-replicate-6.tif', 'C1-p10-20x-zoom-3.1-suse-replicate-7.tif', 'C1-p10-20x-zoom-3.3-suse-replicate-7.tif', 'C1-p10-20x-zoom-3.5-suse-replicate-7.tif', 'C1-p12-20x-zoom-flo-1.4-replicate-8.tif', 'C1-p12-20x-zoom-flo-1.5-replicate-8.tif', 'C1-p12-20x-zoom-flo-1.7-replicate-8.tif', 'C1-p1

### Extract features

Now a for loop will be computed to extract all the features.

In [5]:
# Define array to store extracted features
features_porespy = []

# Loop over the image folder and extract all the features
for i, file_name in enumerate(file_list):
    
    # Load image
    image = imread(images + file_name)
    
    # Predict the image
    prediction = predict_image(image=image, classifier=quapos_lm)
    
    # Rescale the image
    image_rescaled = rescale_image(image=image, voxel_x=0.323, voxel_y=0.323, voxel_z=0.490)
    
    # Rescale the prediction
    prediction_rescaled = rescale_segmentation(segmentation=prediction, voxel_x=0.323, voxel_y=0.323, voxel_z=0.490)
        
    # Feature extraction with porespy regionprops_3D
    features_i = regionprops_3D(prediction_rescaled)
    
    # Define empty array to store features of current image
    extracted_features = []
    
    # Loop over the extracted features to obtain features in accessible data table
    for prop in features_i:
        
        # Obtain some features
        label = prop.label
        surface_area = prop.surface_area
        volume = prop.volume
        bbox_volume = prop.bbox_volume
        convex_volume = prop.convex_volume
        sphericity = prop.sphericity
        solidity = prop.solidity
        
        # Obtain the predicted label in its bounding box, flatten it into 2D and obtain more features
        mask = prop.mask
        mask = cle.maximum_z_projection(mask)
        mask = cle.connected_components_labeling_diamond(mask)
        
        # From 2D label major axis length retrieved
        major_axis_length = regionprops_table(
            label_image = mask,
            properties = ["axis_major_length"])
        major_axis_length = major_axis_length["axis_major_length"][0]
        
        # Minor axis length retrieved
        minor_axis_length = regionprops_table(
            label_image = mask,
            properties = ["axis_minor_length"])
        minor_axis_length = minor_axis_length["axis_minor_length"][0]
        
        # Crofton perimeter retrieved
        perimeter_crofton = regionprops_table(
            label_image = mask,
            properties = ["perimeter_crofton"])
        perimeter_crofton = perimeter_crofton["perimeter_crofton"][0]
        
        # Define names for the different variables
        measurements_dict = {"label": label,
                             "surface_area": surface_area,
                             "volume": volume,
                             "bbox_volume": bbox_volume,
                             "convex_volume": convex_volume,
                             "sphericity": sphericity,
                             "solidity": solidity,
                             "perimeter_2d": perimeter_crofton,
                             "major_axis_length_2d": major_axis_length,
                             "minor_axis_length_2d": minor_axis_length}
        
        # Add features of current image into dataframe
        extracted_features.append(measurements_dict)
    
    # Turn features of current image into a pandas dataframe
    extracted_features = pd.DataFrame(extracted_features)
    
    # Add features of current image into final dataframe
    features_porespy.append(extracted_features)

C:\Users\fsalo\mambaforge\envs\outer-segment-analysis\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6214 qhull input error: not enough points(1) to construct initial simplex (need 4)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1768502921  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _maxoutside  0

  return convex_hull_image(self.image)
C:\Users\fsalo\mambaforge\envs\outer-segment-analysis\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6214 qhull input error: not enough points(3) to construct initial simplex (need 4)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1768502921  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _maxoutside  0

  return convex_hull_image(self.image)
C:\Use

In [6]:
# Concatenate the dataframe
features_porespy = pd.concat(features_porespy)

In [7]:
# Show the features
features_porespy

,label,surface_area,volume,bbox_volume,convex_volume,sphericity,solidity,perimeter_2d,major_axis_length_2d,minor_axis_length_2d
0,1,820.528198,833.0,3456,1525,0.521780,0.546230,57.153073,27.467594,6.307231
1,2,417.350677,754.0,1254,865,0.959910,0.871676,27.791138,10.665223,6.330765
2,3,209.240112,226.0,416,274,0.857521,0.824818,19.421265,7.894590,4.064904
3,4,227.670990,237.0,480,284,0.813471,0.834507,19.421265,8.565180,3.459917
4,5,80.413963,25.0,72,33,0.514177,0.757576,8.599912,3.126944,2.309401
...,...,...,...,...,...,...,...,...,...,...
173,174,1265.061035,1224.0,6732,3094,0.437415,0.395604,93.084047,25.469903,16.027167
174,175,978.182373,1019.0,5250,2200,0.500626,0.463182,67.974426,28.804014,11.992024
175,176,70.511078,44.0,90,54,0.854795,0.814815,12.392149,5.022825,2.508233
176,177,638.787354,473.0,3465,1333,0.459588,0.354839,59.374514,28.204786,8.149047


In [8]:
features_porespy.to_csv("../../measurements/wt-postnatal-development/01-b-feature-extraction-porespy.csv", index = False)